# Download YouTube Dataset
**Tự động download và giải nén**

In [18]:
!pip install kagglehub -q


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import kagglehub
import os

print("📥 Downloading...")
csv_path = kagglehub.dataset_download(
    "rsrishav/youtube-trending-video-dataset",
    path="US_youtube_trending_data.csv"
)
json_path = kagglehub.dataset_download(
    "rsrishav/youtube-trending-video-dataset",
    path="US_category_id.json"
)

📥 Downloading...


In [20]:
import zipfile
import gzip
import shutil

os.makedirs('./data', exist_ok=True)

print("Processing...")

# Detect file type và giải nén
def extract_file(input_path, output_path):
    with open(input_path, 'rb') as f:
        magic = f.read(2)
    
    try:
        # ZIP file (PK)
        if magic == b'PK':
            print("  → Detected: ZIP")
            with zipfile.ZipFile(input_path, 'r') as z:
                names = z.namelist()
                csv_name = [n for n in names if n.endswith('.csv')][0]
                with z.open(csv_name) as zf:
                    with open(output_path, 'wb') as out:
                        shutil.copyfileobj(zf, out)
        # GZIP file
        elif magic == b'\x1f\x8b':
            print("  → Detected: GZIP")
            with gzip.open(input_path, 'rb') as gz:
                with open(output_path, 'wb') as out:
                    shutil.copyfileobj(gz, out)
        # Plain CSV
        else:
            print("  → Detected: Plain file")
            shutil.copy(input_path, output_path)
    except Exception as e:
        print(f"  ⚠️ Error: {e}")
        shutil.copy(input_path, output_path)

extract_file(csv_path, './data/raw_data.csv')
shutil.copy(json_path, './data/category_id.json')


Processing...
  → Detected: ZIP


'./data/category_id.json'

In [26]:
import pandas as pd

df = pd.read_csv('./data/raw_data.csv')
print(f"Dataset: {len(df):,} rows x {len(df.columns)} cols")
print(f"\nColumns:\n{list(df.columns)}\n")
print("First 3 rows:")
df.tail(3)


Dataset: 264,767 rows x 16 cols

Columns:
['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle', 'categoryId', 'trending_date', 'tags', 'view_count', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'description']

First 3 rows:


,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
264764,i6CBnOlJ2_g,foxy hello neighbor,2025-11-18T22:50:42Z,UC14JqA-BgcbceUJT0ytBURQ,Snaspey,20,2025-11-19T10:12:08Z,Five Nights at Freddy's|funny|hello neighbor|f...,83622,8337,0,884,https://i.ytimg.com/vi/i6CBnOlJ2_g/default.jpg,False,False,Fredbear And Foxy Is too much #FNAF #FiveNight...
264765,SKy55H4J23U,Kronos_StyleKumul - Daddy Raus (Official Music...,2025-11-10T14:47:48Z,UC8LMwLpa2Gwae5T1LReNTZw,KronosStyleKumuVEVO,10,2025-11-19T10:12:08Z,Kronos_StyleKumul|BLOODLINE RECORDS|Afrobeat,30342,728,0,37,https://i.ytimg.com/vi/SKy55H4J23U/default.jpg,False,False,Daddy Raus
264766,jfBKSTo67c0,Family Guy 2025 Holiday Special | Official Tra...,2025-11-18T20:59:50Z,UCE5mQnNl8Q4H2qcv4ikaXeA,Hulu,24,2025-11-19T10:12:08Z,hulu|disney|hulu movies|hulu shows|hulu tv|hul...,34178,2265,0,248,https://i.ytimg.com/vi/jfBKSTo67c0/default.jpg,False,False,"This Christmas, Ambitious Career Lady meets Th..."


In [27]:
# Sort data theo trending_date (mới nhất → cũ nhất)
df['trending_date'] = pd.to_datetime(df['trending_date'])
df_sorted = df.sort_values('trending_date', ascending=False)

print("Date Range:")
print(f"  Newest: {df_sorted['trending_date'].iloc[0]}")
print(f"  Oldest: {df_sorted['trending_date'].iloc[-1]}")
print(f"\nTop 5 most recent videos:")
df_sorted.head(5)[['trending_date', 'title', 'channelTitle', 'view_count']]

Date Range:
  Newest: 2025-11-19 10:12:08+00:00
  Oldest: 2020-08-12 00:00:00+00:00

Top 5 most recent videos:


,trending_date,title,channelTitle,view_count
264766,2025-11-19 10:12:08+00:00,Family Guy 2025 Holiday Special | Official Tra...,Hulu,34178
264729,2025-11-19 10:12:08+00:00,Shinedown - Searchlight (Official Video),Shinedown,93333
264739,2025-11-19 10:12:08+00:00,Wake Up Dead Man: A Knives Out Mystery | Offic...,Netflix,3246810
264738,2025-11-19 10:12:08+00:00,Fine Chiki,JNR VIGI - Topic,83413
264737,2025-11-19 10:12:08+00:00,I Built A Brainrot AUTO PRINTER To Troll My Fr...,Doodle and Arkey,110069
